# Initialization
---

### Loading the data

In [ ]:
df1 = spark.read.format('parquet').load("dbfs:/FileStore/tables/")

In [ ]:
df1.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- timedate: timestamp (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- EncodedLinkPoints: string (nullable = true)
 |-- EncodedPolyLineLvls: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- TranscomId: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [ ]:
df1 = df1.withColumnRenamed("Speed", "y")
df1 = df1.withColumnRenamed("timedate", "ds")
df1 = df1.withColumnRenamed("LinkId", "link_id")

In [ ]:
from pyspark.sql.types import StringType
df1 = df1.withColumn("link_id",df1.link_id.cast('string'))

In [ ]:
df = df1.select('ds', 'link_id', 'y')

In [ ]:
from pyspark.sql.functions import count, col

# Filtering link points with atleast 200,000 records.
list = df.groupBy('link_id').agg(count('link_id').alias('records')).filter(col('records')>200000).select('link_id')

#storing these Link Points to a list 
list = list.toPandas().link_id.tolist()

#specifying filters to clean the data
cleaning_filters = ( df.link_id.isin(list) ) & (df.ds >= '2018-01-01') & (df.ds < '2022-01-01') & (df.y < 68) & (df.y > 0)

In [ ]:
# Filtering rows to clean the data
df = df.filter( cleaning_filters )

In [ ]:
df.cache()
df.printSchema()
df.show(10)

root
-- ds: timestamp (nullable = true)
-- link_id: string (nullable = true)
-- y: double (nullable = true)

+-------------------+-------+-----+
 ds|link_id| y|
+-------------------+-------+-----+
2021-12-07 22:58:03|4329507|16.77|
2021-12-07 22:58:03|4362244|45.98|
2021-12-07 22:58:03|4362247|41.63|
2021-12-07 22:58:03|4362249|24.85|
2021-12-07 22:58:03|4362250|49.08|
2021-12-07 22:58:03|4362251|27.34|
2021-12-07 22:58:03|4362252|38.52|
2021-12-07 22:58:03|4362314|40.38|
2021-12-07 22:58:03|4362342| 9.32|
2021-12-07 22:58:03|4456479|39.14|
+-------------------+-------+-----+
only showing top 10 rows

In [ ]:
df.createOrReplaceTempView('data')

# Time-Seriers Analysis | Entire Dataset
---

## Basic Time-Series Data Exploration on entire dataset

#### Overall Trend

In [ ]:
%sql
 
SELECT DATE_TRUNC('day', ds) AS Day_Year, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY Day_Year
ORDER BY Day_Year;

Day_Year,Average_Traffic_Speed
2018-01-01T00:00:00.000+0000,43.91112507721048
2018-01-02T00:00:00.000+0000,41.98282428907808
2018-01-03T00:00:00.000+0000,39.60507643745752
2018-01-04T00:00:00.000+0000,34.17475279619008
2018-01-05T00:00:00.000+0000,38.19790973395499
2018-01-06T00:00:00.000+0000,41.798857738582754
2018-01-07T00:00:00.000+0000,43.34069685039667
2018-01-08T00:00:00.000+0000,38.608231897936406
2018-01-09T00:00:00.000+0000,37.575682945280334
2018-01-10T00:00:00.000+0000,38.83931359728577


In [ ]:
%sql

SELECT DATE_TRUNC('MM', ds) AS Month_Years, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY Month_Years
ORDER BY Month_Years;

Month_Years,Average_Traffic_Speed
2018-01-01T00:00:00.000+0000,40.58938122468354
2018-02-01T00:00:00.000+0000,41.4682304182476
2018-03-01T00:00:00.000+0000,39.96689915679835
2018-04-01T00:00:00.000+0000,39.41342617590727
2018-05-01T00:00:00.000+0000,38.5426343655323
2018-06-01T00:00:00.000+0000,38.182832224740444
2018-07-01T00:00:00.000+0000,39.54546326140686
2018-08-01T00:00:00.000+0000,38.803027674608344
2018-09-01T00:00:00.000+0000,38.88729869028744
2018-10-01T00:00:00.000+0000,36.93128860367636


#### Annual Pattern

In [ ]:
%sql

SELECT DATEADD(DAYOFYEAR, DAYOFYEAR(ds), '1999-12-31') day_month, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY day_month
ORDER BY day_month;

day_month,Average_Traffic_Speed
2000-01-01T00:00:00.000+0000,44.17477441123609
2000-01-02T00:00:00.000+0000,42.57051808574029
2000-01-03T00:00:00.000+0000,41.85423593248462
2000-01-04T00:00:00.000+0000,40.71699900582258
2000-01-05T00:00:00.000+0000,42.02113246742318
2000-01-06T00:00:00.000+0000,42.22369579608318
2000-01-07T00:00:00.000+0000,41.883757844788555
2000-01-08T00:00:00.000+0000,40.47119405375261
2000-01-09T00:00:00.000+0000,40.3390435506515
2000-01-10T00:00:00.000+0000,40.97276139053595


In [ ]:
%sql

SELECT MONTH(ds) AS Month, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY Month
ORDER BY Month;

Month,Average_Traffic_Speed
1,41.31928413696354
2,40.714576884251315
3,40.68334415848797
4,41.444758776327156
5,39.90739231592663
6,38.84827024906913
7,39.63809152612314
8,39.43541174631792
9,39.035663579976436
10,38.408387833456224


In [ ]:
%sql

SELECT DATEADD(DAYOFYEAR, DAYOFYEAR(ds), '1999-12-31') day_month, YEAR(ds) AS Year, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY day_month, Year
ORDER BY day_month;

day_month,Year,Average_Traffic_Speed
2000-01-01T00:00:00.000+0000,2018,43.91112507721048
2000-01-01T00:00:00.000+0000,2019,42.769250704067375
2000-01-01T00:00:00.000+0000,2021,46.655596678440084
2000-01-01T00:00:00.000+0000,2020,43.95899364350333
2000-01-02T00:00:00.000+0000,2020,42.31380287019403
2000-01-02T00:00:00.000+0000,2018,41.98282428907808
2000-01-02T00:00:00.000+0000,2019,42.59527962166596
2000-01-02T00:00:00.000+0000,2021,43.983660393097374
2000-01-03T00:00:00.000+0000,2019,41.96993632375968
2000-01-03T00:00:00.000+0000,2021,45.89557614880626


In [ ]:
%sql

SELECT YEAR(ds) AS Year, Month(ds) AS Month, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY Year, Month
ORDER BY Year, Month;

Year,Month,Average_Traffic_Speed
2018,1,40.58938122468354
2018,2,41.4682304182476
2018,3,39.96689915679835
2018,4,39.41342617590727
2018,5,38.5426343655323
2018,6,38.182832224740444
2018,7,39.54546326140686
2018,8,38.803027674608344
2018,9,38.88729869028744
2018,10,36.93128860367636


#### Weekly Pattern

In [ ]:
%sql

SELECT DATEADD(HOUR, HOUR(ds), DATEADD(DAY, DAYOFWEEK(ds), '2000-09-30') ) hour_day, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY hour_day
ORDER BY hour_day;

hour_day,Average_Traffic_Speed
2000-10-01T00:00:00.000+0000,37.886610104585095
2000-10-01T01:00:00.000+0000,39.582207225280655
2000-10-01T02:00:00.000+0000,40.90479837705186
2000-10-01T03:00:00.000+0000,41.67564529137384
2000-10-01T04:00:00.000+0000,42.78942273903026
2000-10-01T05:00:00.000+0000,44.00244839507833
2000-10-01T06:00:00.000+0000,44.723808611674784
2000-10-01T07:00:00.000+0000,45.21568657565793
2000-10-01T08:00:00.000+0000,46.05464285432702
2000-10-01T09:00:00.000+0000,47.73725707732286


In [ ]:
%sql

SELECT DAYOFWEEK(ds) day, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY day
ORDER BY day;

day,Average_Traffic_Speed
1,42.06554832152207
2,40.00259880206134
3,39.67485028208052
4,39.49248576676379
5,38.69277350807812
6,38.29730176871614
7,40.922854496740015


In [ ]:
%sql

SELECT DATEADD(HOUR, HOUR(ds), DATEADD(DAY, DAYOFWEEK(ds), '2000-09-30') ) day, YEAR(ds) AS Year, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY day, Year
ORDER BY day;

day,Year,Average_Traffic_Speed
2000-10-01T00:00:00.000+0000,2020,41.890061086784094
2000-10-01T00:00:00.000+0000,2018,36.7525613520075
2000-10-01T00:00:00.000+0000,2021,36.23040998325951
2000-10-01T00:00:00.000+0000,2019,36.89841721183922
2000-10-01T01:00:00.000+0000,2021,38.24537062335935
2000-10-01T01:00:00.000+0000,2019,38.4932472201065
2000-10-01T01:00:00.000+0000,2020,43.25017433257345
2000-10-01T01:00:00.000+0000,2018,38.502078293170335
2000-10-01T02:00:00.000+0000,2018,39.82021209182875
2000-10-01T02:00:00.000+0000,2019,39.84235084524916


In [ ]:
%sql

SELECT DAYOFWEEK(ds) day, YEAR(ds) AS Year, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY day, Year
ORDER BY day;

day,Year,Average_Traffic_Speed
1,2018,41.20313601772859
1,2021,40.71421552853373
1,2019,41.10421533659881
1,2020,45.307105717797064
2,2018,39.49738497456082
2,2021,38.54063461083529
2,2020,42.98819971881964
2,2019,39.060387882754156
3,2019,38.737962828575
3,2021,38.56424176565951


#### Daily Pattern

In [ ]:
%sql

SELECT (MINUTE(ds) + HOUR(ds) * 60) / 60 AS minute_hour, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY minute_hour
ORDER BY minute_hour;

minute_hour,Average_Traffic_Speed
0.0,35.65917203311868
0.016666666666666666,35.29634819532909
0.03333333333333333,35.90856479982731
0.05,37.62375195166055
0.06666666666666667,39.37752701403564
0.08333333333333333,35.972781774580326
0.1,37.65969968051119
0.11666666666666667,36.243013463021185
0.13333333333333333,37.963590784863
0.15,39.57728235567953


In [ ]:
%sql

SELECT HOUR(ds) AS Hour, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY Hour
ORDER BY Hour;

Hour,Average_Traffic_Speed
0,38.89486799231258
1,40.82305428054444
2,42.022674294233084
3,42.78749343664169
4,43.50631492705456
5,44.35660336713423
6,45.262096532287636
7,46.38586767861874
8,47.75771166254878
9,48.08047113283098


In [ ]:
%sql

SELECT (MINUTE(ds) + HOUR(ds) * 60) / 60 AS minute_hour, YEAR(ds) AS year, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY minute_hour, year
ORDER BY minute_hour, year;

minute_hour,year,Average_Traffic_Speed
0.0,2018,35.13841530054641
0.0,2019,32.78990740740739
0.0,2021,40.17758241758243
0.016666666666666666,2018,33.31271739130433
0.016666666666666666,2019,34.99754442649434
0.016666666666666666,2020,34.951142857142855
0.016666666666666666,2021,37.232755102040805
0.03333333333333333,2018,36.01062884666847
0.03333333333333333,2019,35.09833409891277
0.03333333333333333,2020,39.16956740865182


In [ ]:
%sql

SELECT HOUR(ds) AS Day, YEAR(ds) AS Year, AVG(y) AS Average_Traffic_Speed
FROM data
GROUP BY Day, Year
ORDER BY Day, Year;

Day,Year,Average_Traffic_Speed
0,2018,37.98094620394083
0,2019,37.60693695234149
0,2020,42.44612604115106
0,2021,37.736582681265375
1,2018,40.06671660672795
1,2019,39.65872040296239
1,2020,43.80105995015623
1,2021,39.95295711786405
2,2018,41.35660568681088
2,2019,41.02068101922347
